In [1]:
import os
import time
import datetime
import numpy as np
import pandas as pd

## 第一步：加载合同报表内容到内存

In [2]:
# 参数
PATH = os.path.abspath(r"C:\Users\Administrator\Downloads")
CONTRACT_FILENAME = "7月合同报表.xls"

## 宽厚板指数销量和价格获取
MONTH_ZHONGBAN_COUNT = 77150
ZHONGBAN_PRICE = int(input("请输入中板均价："))

请输入中板均价：3879


In [3]:
# 读取当月月报
df = pd.DataFrame(pd.read_excel(os.path.join(PATH,CONTRACT_FILENAME),header =1))
# df.head()

companys = df['卖家'].unique().tolist()

for co in companys:
    if co.__contains__("淮钢"):
        print("今天又淮钢数据，模板中没有，是否需要统计，请教严丰")

## 第2步：转换df的“合同日期”时间，用于计算当周的数据（如上周五-本周四，或周一到周五），通过手动输入设置

In [4]:
def format_date(test_date_string):
    date_value = datetime.datetime.strptime(test_date_string,"%Y-%m-%d %H:%M:%S")
    contract_year = date_value.date().year
    contract_month = date_value.date().month
    contract_day = date_value.date().day
    return str(contract_year)+ "-" + str(contract_month) + "-" + str(contract_day)

df["合同日期"] = df["合同日期"].map(format_date)

In [5]:
start_day = input("请输入统计周的开始日期值：")  # 输入格式如：2020-7-3
end_day = input("请输入统计周的结束日期值：")
def get_week(day):
    start_strptime = datetime.datetime.strptime(start_day, '%Y-%m-%d')
    end_strptime = datetime.datetime.strptime(end_day, '%Y-%m-%d')
    
    day_strptime = datetime.datetime.strptime(day, '%Y-%m-%d')
    if day_strptime >= start_strptime and day_strptime <= end_strptime:
        return True
    else:
        return False


df["是否统计周"] = df["合同日期"].map(get_week)

请输入统计周的开始日期值：2020-7-3
请输入统计周的结束日期值：2020-7-9


## 第3步：根据“是否是统计周”字段进行条件筛选，将获取到的当天的数据进行汇总，求和，取出当天的计算数据。

## 统计周数据
- 当周 平台钢材成交总量   **steel_transaction_count**
    - 当周 合同【数量】字段汇总
    - 当周 中板指数成交数量（根据月度指标求均值，或问严丰要实际数据）
- 当周现货竞拍成交总量   **bids_transaction_count**
        - 当周物贸余材竞拍成交量 + 当周轧花厂竞拍成交量 + 当周东北特钢竞拍成交量
    - 物贸余材竞拍成交总量 **wumao_bids_transaction_count**
        - 卖家是【江苏沙钢物资贸易有限公司】的【数量】字段汇总
    - 轧花厂竞拍成交总量 **yahuacang_bids_transaction_count**
        - 卖家是【张家港市锦丰轧花剥绒有限责任公司】的【数量】字段汇总
    - 东北特钢竞拍成交总量  **dongbei_bids_transaction_count**
        - 卖家是【东北特殊钢集团股份有限公司】的【数量】字段汇总
- 期货挂牌成交 **qihuo_guapai_count**
    - 卖家是【江苏沙钢物资贸易有限公司1】，产品是["优线", "普冷","酸洗","轧硬"]
- 期货撮合成交 **qihuo_cuohe_count**
    - 卖家是【江苏沙钢物资贸易有限公司1】，产品是["花纹卷", "普卷","C料","螺纹","盘螺"]
- 现货竞拍增值
- 平均吨钢竞拍增值

- 当周 平台钢材成交总量 **month_steel_transaction_count**
    - 竞拍成交总量   **month_bids_transaction_count**
    - 期货挂牌成交总量  **month_qihuo_transaction_count**
    - 竞拍撮合成交总量  **month_jingpai_cuohe_transaction_count**
    - 轧花厂成交总量   **month_yahuacang_transaction_count**


In [6]:
bids_transaction_count = 0

# 物贸余材竞拍成交总量 wumao_bids_transaction_count，卖家是【江苏沙钢物资贸易有限公司】的【数量】字段汇总
wumao_bids_transaction_count = df["数量"].loc[(df["是否统计周"] == True) & (df["卖家"] == "江苏沙钢物资贸易有限公司")].sum()
wumao_bids_transaction_price = df["贷款"].loc[(df["是否统计周"] == True) & (df["卖家"] == "江苏沙钢物资贸易有限公司")].sum()
# 轧花厂竞拍成交总量 yahuacang_bids_transaction_count，卖家是【张家港市锦丰轧花剥绒有限责任公司】的【数量】字段汇总
yahuacang_bids_transaction_count = df["数量"].loc[(df["是否统计周"] == True) & (df["卖家"] == "张家港市锦丰轧花剥绒有限责任公司")].sum()
yahuacang_bids_transaction_price = df["贷款"].loc[(df["是否统计周"] == True) & (df["卖家"] == "张家港市锦丰轧花剥绒有限责任公司")].sum()
# 东北特钢竞拍成交总量  dongbei_bids_transaction_count，卖家是【东北特殊钢集团股份有限公司】的【数量】字段汇总
dongbei_bids_transaction_count = df["数量"].loc[(df["是否统计周"] == True) & (df["卖家"] == "东北特殊钢集团股份有限公司")].sum()
dongbei_bids_transaction_price = df["贷款"].loc[(df["是否统计周"] == True) & (df["卖家"] == "东北特殊钢集团股份有限公司")].sum()


# 当周现货竞拍成交总量  bids_transaction_count，当周物贸余材竞拍成交量 + 当周轧花厂竞拍成交量 + 当周东北特钢竞拍成交量
bids_transaction_count = wumao_bids_transaction_count + yahuacang_bids_transaction_count + dongbei_bids_transaction_count
# 当周现货竞拍成交总额 bids_transaction_price，当周物贸余材竞拍成交总额 + 当周轧花厂竞拍成总额  + 当周东北特钢竞拍成总额 
bids_transaction_price = wumao_bids_transaction_price + yahuacang_bids_transaction_price + dongbei_bids_transaction_price

"1、本周完成钢材在线交易3.53万吨，成交金额1.33亿元，招标采购标书129份标书定标，中标金额约为3.69亿元，另有680份标书进行中；新增会员23家，其中销售类会员5家，供应类会员18家；
2、本周平台完成现货竞拍1.6万吨，其中沙钢物贸成交1.46万吨，轧花厂成交1390吨，东北特钢成交53吨，宽厚板指数定价成交1.93万吨，本周竞拍增值73.16万元，吨钢增值50.11元。"


In [7]:
# 期货挂牌成交 qihuo_guapai_count，卖家是【江苏沙钢物资贸易有限公司1】，产品是["优线", "普冷","酸洗","轧硬"]
# 期货撮合成交 qihuo_cuohe_count，卖家是【江苏沙钢物资贸易有限公司1】，产品是["花纹卷", "普卷","C料","螺纹","盘螺"]
guapai_week = df.query('品名 == ["优线", "普冷","酸洗","轧硬"]').loc[(df["卖家"] == "江苏沙钢物资贸易有限公司1") & (df["是否统计周"] == True)]
qihuo_guapai_count = guapai_week["数量"].sum()
cuohe_week = df.query('品名 == ["花纹卷", "普卷","C料","螺纹","盘螺"]').loc[(df["卖家"] == "江苏沙钢物资贸易有限公司1") & (df["是否统计周"] == True)]
qihuo_cuohe_count =  cuohe_week["数量"].sum()

## 中厚板成交量和价格

In [8]:
week_zhongban_count = 77150 / 4 
week_zhongban_price = week_zhongban_count * ZHONGBAN_PRICE

## 周报内容拼接

In [9]:
biaoshu_confirm = int(input("请输入本周招标采购标书定标份数："))
biaoshu_money = input("请输入本周招标采购中标金额：")
biaoshu_continue = int(input("请输入本周正在进行中的标书："))

increase_count = int(input("请输入本周新增会员数："))
increase_count_sales = int(input("请输入销售类会员数："))
increase_count_gongying = int(input("请输入供应类会员数："))

请输入本周招标采购标书定标份数：129
请输入本周招标采购中标金额：3.69
请输入本周正在进行中的标书：680
请输入本周新增会员数：23
请输入销售类会员数：5
请输入供应类会员数：18


In [10]:
week_report1 = """1、本周完成钢材在线交易{0}万吨，成交金额{1}亿元；
招标采购标书{2}份标书定标，中标金额约为{3}亿元，另有{4}份标书进行中；
新增会员{5}家，其中销售类会员{6}家，供应类会员{7}家；
""".format(
           round((bids_transaction_count + week_zhongban_count) / 10000  ,2),
           round((bids_transaction_price + week_zhongban_price) / 10000 / 10000 ,2),
           round(biaoshu_confirm,2),
           biaoshu_money,
           round(biaoshu_continue,2),
           round(increase_count,2),
           round(increase_count_sales,2),
           round(increase_count_gongying,2),
          )

In [15]:
# 本周内容拼接
week_report2 = """2、本周平台完成现货竞拍{0}万吨，
其中沙钢物贸成交{1}吨，
轧花厂成交{2}吨，
东北特钢成交{3}吨，
宽厚板指数定价成交{4}万吨，
本周竞拍增值()万元，
吨钢增值()元。""".format(
           round(bids_transaction_count / 10000,2),
           round(wumao_bids_transaction_count,2),
           round(yahuacang_bids_transaction_count,2),
           round(dongbei_bids_transaction_count,2),
           round(week_zhongban_count / 10000,2),
          )

In [16]:
week_report1 + week_report2

'1、本周完成钢材在线交易3.53万吨，成交金额1.35亿元；\n招标采购标书129份标书定标，中标金额约为3.69亿元，另有680份标书进行中；\n新增会员23家，其中销售类会员5家，供应类会员18家；\n2、本周平台完成现货竞拍1.6万吨，\n其中沙钢物贸成交14602.03吨，\n轧花厂成交1390.0吨，\n东北特钢成交53.38吨，\n宽厚板指数定价成交1.93万吨，\n本周竞拍增值()万元，\n吨钢增值()元。'

"1、本周完成钢材在线交易3.53万吨，成交金额1.33亿元，招标采购标书129份标书定标，中标金额约为3.69亿元，另有680份标书进行中；新增会员23家，其中销售类会员5家，供应类会员18家； 2、本周平台完成现货竞拍1.6万吨，其中沙钢物贸成交1.46万吨，轧花厂成交1390吨，东北特钢成交53吨，宽厚板指数定价成交1.93万吨，本周竞拍增值73.16万元，吨钢增值50.11元。"